In [6]:
# Download HTML da página de estatísticas
import requests

scraping_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(scraping_url)

In [7]:
# Scraping
from bs4 import BeautifulSoup

soup = BeautifulSoup(data.text)
stats_table = soup.select('table.stats_table')[0]

links = stats_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

team_urls = [f"https://fbref.com{l}" for l in links]

In [8]:
team_url = team_urls[0]
data = requests.get(team_url)

In [9]:
import pandas as pd
from io import StringIO

matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")
matches[0].head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (4),1 (1),Manchester City,NaN,NaN,45.0,81145.0,Martin Ødegaard,4-3-3,Stuart Attwell,Match Report,Arsenal won on penalty kicks following normal ...
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2,1,Nott'ham Forest,0.8,1.2,78.0,59984.0,Martin Ødegaard,4-3-3,Michael Oliver,Match Report,NaN
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1,0,Crystal Palace,2.0,1.0,53.0,24189.0,Martin Ødegaard,4-3-3,David Coote,Match Report,NaN
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2,2,Fulham,3.2,0.6,71.0,59961.0,Martin Ødegaard,4-3-3,Paul Tierney,Match Report,NaN
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3,1,Manchester Utd,2.3,0.9,55.0,60192.0,Martin Ødegaard,4-3-3,Anthony Taylor,Match Report,NaN


In [10]:
soup = BeautifulSoup(data.text)

links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [11]:
data = requests.get(f"https://fbref.com{links[0]}")
shooting = pd.read_html(StringIO(data.text), match="Shooting")[0]
shooting.head()

For Arsenal                                                              \
         Date   Time              Comp                Round  Day    Venue   
0  2023-08-06  16:00  Community Shield  FA Community Shield  Sun  Neutral   
1  2023-08-12  12:30    Premier League          Matchweek 1  Sat     Home   
2  2023-08-21  20:00    Premier League          Matchweek 2  Mon     Away   
3  2023-08-26  15:00    Premier League          Matchweek 3  Sat     Home   
4  2023-09-03  16:30    Premier League          Matchweek 4  Sun     Home   

                                         ... Standard               Expected  \
  Result     GF     GA         Opponent  ...     Dist   FK PK PKatt       xG   
0      D  1 (4)  1 (1)  Manchester City  ...      NaN  NaN  0     0      NaN   
1      W      2      1  Nott'ham Forest  ...     19.1  0.0  0     0      0.8   
2      W      1      0   Crystal Palace  ...     16.4  0.0  1     1      2.0   
3      D      2      2           Fulham  ...     13.8  0.0  1     1      3.2   
4      W      3      1   Manchester Utd  ...     15.0  0.0  0     0      2.3   

                            Unnamed: 25_level_0  
  npxG npxG/Sh G-xG np:G-xG        Match Report  
0  NaN     NaN  NaN     NaN        Match Report  
1  0.8    0.06  1.2     1.2        Match Report  
2  1.2    0.09 -1.0    -1.2        Match Report  
3  2.4    0.14 -1.2    -1.4        Match Report  
4  2.3    0.13  0.7     0.7        Match Report  

[5 rows x 26 columns]

In [12]:
shooting.columns = shooting.columns.droplevel()
shooting.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (4),1 (1),Manchester City,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2,1,Nott'ham Forest,...,19.1,0.0,0,0,0.8,0.8,0.06,1.2,1.2,Match Report
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1,0,Crystal Palace,...,16.4,0.0,1,1,2.0,1.2,0.09,-1.0,-1.2,Match Report
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2,2,Fulham,...,13.8,0.0,1,1,3.2,2.4,0.14,-1.2,-1.4,Match Report
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3,1,Manchester Utd,...,15.0,0.0,0,0,2.3,2.3,0.13,0.7,0.7,Match Report


In [13]:
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [14]:
years = list(range(2024, 2022, -1))
years

[2024, 2023]

In [15]:
import time

all_matches = []

for year in years:
    data = requests.get(scraping_url)
    soup = BeautifulSoup(data.text)    
    stats_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in stats_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")

        data = requests.get(team_url)
        matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]

        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(StringIO(data.text), match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue

        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(2)